In [1]:
%load_ext autoreload
%autoreload 2

import ipywidgets as widgets
import threading
import numpy as np
import os
from visualiser.frame_cache import *
from ipywidgets import *
from visualiser.train_progress_bar import set_progress_to_zero, get_object
from train import train

training_thread = threading.Thread(target=train, args=('visualiser/created_experiment.json',))

def get_new_experiment_id():
    id = 0
    while os.path.exists(os.path.join('experiments',str(id))):
        id += 1
    print(f"New experiment id:{id}")
    return id

train_button = widgets.Button(description='Start Training',disabled=False,button_style='')

def wait_for_train_finished():
    training_thread.join()
    # train_button.disabled = False
    # train_button.description = 'Restart Training'
    from IPython.display import display
    def _show(x):
        display(get_image_of_frame_at(x))
    frame_slider = widgets.IntSlider(min=0, max=get_total_frame_number() - 1, value=0)
    frame_player = widgets.Play(
        value=0,
        min=0,
        max=get_total_frame_number() - 1,
        step=1,
        interval=500,
        description="Press play",
        disabled=False
    )
    frame_input_ui = widgets.HBox([frame_player, frame_slider])
    widgets.jslink((frame_slider, 'value'), (frame_player, 'value'))
    img_output = widgets.interactive_output(_show, {'x': frame_slider})
    img_ui = widgets.VBox([frame_input_ui, img_output])
    display(img_ui)
    
wait_for_training_thread = threading.Thread(target=wait_for_train_finished)

def train_button_clicked(c):
    train_button.disabled = True
    set_json_file()
    set_progress_to_zero()
    training_thread.start()
    wait_for_training_thread.start()

train_button.on_click(train_button_clicked)

from rewards import rewards
reward_function_vars = np.array(dir(rewards))
reward_function_names = reward_function_vars[np.char.endswith(reward_function_vars,"reward")]

total_episode_textbox = widgets.BoundedIntText(value=1,min=1,max=999,step=1,description='Number of episodes (years):',style={'description_width':'initial'})
steps_per_frame_save_textbox = widgets.BoundedIntText(value=200,min=1,max=365*24-1,step=1,description='Steps per frame save:',style={'description_width':'initial'})
reward_function_dropdown = widgets.Dropdown(options=reward_function_names, value='default_reward',description='Reward function:',style={'description_width':'initial'})

import json
def set_json_file():
    with open('visualiser/created_experiment.json', 'r') as f:
        json_data = json.load(f)
        json_data['id'] = get_new_experiment_id()
        json_data['episodes'] = total_episode_textbox.value
        json_data['steps_per_frame_save'] = steps_per_frame_save_textbox.value
        json_data['reward'] = reward_function_dropdown.value
    with open('visualiser/created_experiment.json', 'w') as f:
        f.write(json.dumps(json_data))

train_progress_bar = get_object()
set_progress_to_zero()
experiment_input = widgets.VBox([total_episode_textbox, steps_per_frame_save_textbox, reward_function_dropdown])
experiment_ui = widgets.VBox([experiment_input, train_button, train_progress_bar])
display(experiment_ui)

New experiment id:2
Loading agent <class 'agents.agents.ddpg.DDPGAgent'>
Setting reward <function default_reward at 0x0000018346775790>
Setting environment ./data/citylearn_challenge_2022_phase_1/schema.json
